In [1]:
import huggingface_hub

huggingface_hub.login("Your_Huggingface_Token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from huggingface_hub import snapshot_download

model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

llm = LLM(model=model_id, enable_lora=True, max_lora_rank=256)

INFO 09-14 23:45:03 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='allganize/Llama-3-Alpha-Ko-8B-Instruct', speculative_config=None, tokenizer='allganize/Llama-3-Alpha-Ko-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=allganize/Llama-3-Alpha-Ko-8B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1,

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-14 23:45:10 model_runner.py:1008] Loading model weights took 14.9634 GB
INFO 09-14 23:45:28 gpu_executor.py:122] # GPU blocks: 27190, # CPU blocks: 2048
INFO 09-14 23:45:34 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-14 23:45:34 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-14 23:45:55 model_runner.py:1430] Graph capturing finished in 20 secs.


In [3]:
sampling_params_lora1 = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=50)
lora_adapter1 = "daje/chapter5_psychological_chatbots"
lora_adapter1_path = snapshot_download(repo_id=lora_adapter1)
lora1 = LoRARequest("lora1", 1, lora_adapter1_path)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_1578/1501373901.py:4: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora1 = LoRARequest("lora1", 1, lora_adapter1_path)


In [4]:
sampling_params_lora2 = SamplingParams(temperature=0.1, max_tokens=50)
lora_adapter2 = "daje/chapter5_code-llama3-8B-text-to-sql-ver0.1"
lora_adapter2_path = snapshot_download(repo_id=lora_adapter2)
lora2 = LoRARequest("lora2", 2, lora_adapter2_path)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

/tmp/ipykernel_1578/1020445224.py:4: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora2 = LoRARequest("lora2", 2, lora_adapter2_path)


In [5]:
prompts_lora1 = [
    "일요일인데 새벽6시에 일어났어 ㅜㅜ",
    "요즘 대상포진이 걸려서 고생했어",
]

outputs = llm.generate(prompts_lora1, sampling_params_lora1, lora_request=lora1)

for output in outputs:
    generated_text = output.outputs[0].text
    print(generated_text)
    print('------')

Processed prompts: 100%|██████████| 2/2 [01:13<00:00, 36.77s/it, est. speed input: 0.39 toks/s, output: 1.36 toks/s]

. 평소에는 매일 일어나는 것이 힘들었는데 요즘은 정말 힘들어요. 아침에 일어나기 싫은 마음이 너무 커서 밤에도 잠을 잘 못자고, 주말이 되면 정말 기
------
. 이제는 매일 밤마다 눈이 따가워지고, 눈 앞이 흐려지고, 눈이 불편해져서 일도 제대로 못하고, 집에서도 편하게 쉬지도 못하고, 잠도 못자고,
------


In [6]:
prompts_lora2 = [
    """Task:최고 총액을 말해줘.'
SQL table: CREATE TABLE table_12014 (
    "Rider" text,
    "Horse" text,
    "Faults" text,
    "Round 1 + 2A Points" text,
    "Total" real
)
SQL query:""",
    "sql로 평균 구하는거 알려줘.",
]

outputs = llm.generate(prompts_lora2, sampling_params_lora2, lora_request=lora2)

for output in outputs:
    generated_text = output.outputs[0].text
    print(generated_text)
    print('------')

Processed prompts: 100%|██████████| 2/2 [00:48<00:00, 24.44s/it, est. speed input: 0.45 toks/s, output: 2.05 toks/s]

 그리고 그룹화도 해줘.
SQL table: CREATE TABLE table_203_203 (
    id number,
    "rank" number,
    "name" text,
    "nationality" text,
    "result" text,
   
------
 그리고 이름을 내림차순으로 정렬해줘.
SQL table: CREATE TABLE table_203_203 (
    id number,
    "rank" number,
    "name" text,
    "nationality" text,
    "
------
